In [3]:
!test -d test || (wget -O test.zip https://zenodo.org/records/8437121/files/marmoset-dlc-2021-05-07.zip?download=1 && unzip -q test.zip -d ./test && rm -f test.zip)
!wget -O sam_vit_h.pth https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install ultralytics

--2024-09-09 07:14:14--  https://dl.fbaipublicfiles.com/segment_anything/sam_vit_h_4b8939.pth
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.172.202.93, 18.172.202.63, 18.172.202.54, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.172.202.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2564550879 (2.4G) [binary/octet-stream]
Saving to: ‘sam_vit_h.pth’

sam_vit_h.pth       100%[===================>]   2.39G  61.9MB/s    in 39s     

2024-09-09 07:14:54 (62.0 MB/s) - ‘sam_vit_h.pth’ saved [2564550879/2564550879]

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-psc7mnpx
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-psc7mnpx
  Resolved https://github.com/facebookresearch/segment-anything.git to commit 526fd066dea338ba2ca08886853bd37ffd6a8aec
  Preparing metadata (setup.py) ... done


In [44]:
from ultralytics import YOLO
import glob
import numpy as np
import torch
import cv2
import os
from tqdm import tqdm
from segment_anything import SamPredictor, sam_model_registry

class Marmo:
    COLORS = [(255, 255, 0), (255, 0, 255), (0, 255, 255),(255, 0, 0), (0, 255, 0), (0, 0, 255)]
    # SKELETON = [(0, 1), (0, 2), (0, 3), (3, 7), (7, 8), (5, 12), (13, 14), (11, 14), (3, 9), (9, 10), (3, 4), (4, 5), (6, 13), (6, 11)]
    CONFIDENCE_THRESHOLD = 0.5

    def __init__(self, pose_model_path, sam_checkpoint_path):
        self.pose_model = YOLO(pose_model_path)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        sam = sam_model_registry["default"](checkpoint=sam_checkpoint_path).to(device)
        self.mask_predictor = SamPredictor(sam)

    @staticmethod
    def apply_color_masks(image, masks):
        for i, mask in enumerate(masks):
            colored_mask = cv2.cvtColor((mask[0].squeeze() * 255).astype('uint8'), cv2.COLOR_GRAY2BGR)
            color_overlay = np.zeros_like(image)
            color_overlay[:] = Marmo.COLORS[i]
            color_overlay[colored_mask == 0] = 0
            image = cv2.addWeighted(image, 0.7, color_overlay, 0.3, 0)
        return image
    
    @staticmethod
    def draw_joint(image, points):
        for i in range(len(points)):
            for x, y in points[i]:
                image = cv2.circle(image, (int(x), int(y)), 2, Marmo.COLORS[i+1], -1)
        return image

    # @staticmethod
    # def draw_skeleton(image, points, masks):
    #     overlay = image.copy()
    #     for i, mask in enumerate(masks):
    #         for start_i, end_i in Marmo.SKELETON:
    #             if mask[start_i] and mask[end_i]:
    #                 start_point = tuple(points[i][start_i].astype(int))
    #                 end_point = tuple(points[i][end_i].astype(int))
    #                 cv2.line(overlay, start_point, end_point, Marmo.COLORS[i], 2)
    #     return cv2.addWeighted(image, 0.3, overlay, 0.7, 0)

    def predict_image(self, image, mode="points"):
        if mode not in ["points", "box"]:
            raise ValueError(f"Invalid mode of operation: use {' or '.join(['points', 'box'])}")

        pose_result = self.pose_model(image)[0]
        if pose_result is None or pose_result.keypoints is None or pose_result.keypoints.conf is None:
            return image

        joint_points = pose_result.keypoints.xy.cpu().numpy()
        boxes = pose_result.boxes.xyxy.cpu().numpy()
        point_confidences = pose_result.keypoints.conf.cpu().numpy()
        confidence_mask = point_confidences > self.CONFIDENCE_THRESHOLD
        zero_mask = ~np.all(joint_points == 0, axis=2)
        point_mask = zero_mask & confidence_mask
        input_points = np.where(np.expand_dims(point_mask, axis=2), joint_points, 0)

        self.mask_predictor.set_image(image)

        if mode == "points":
            sam_masks = [self.mask_predictor.predict(point_coords=point, point_labels=label.astype(int), multimask_output=False) for point, label in zip(input_points, point_mask)]
        else:  # mode "box"
            sam_masks = [self.mask_predictor.predict(box=box, multimask_output=False) for box in boxes]

        image = self.apply_color_masks(image, sam_masks)
        # image = self.draw_skeleton(image, input_points, sam_masks)
        return self.draw_joint(image, input_points)

    def predict_images(self, image_paths, save_path, mode="points"):
        os.makedirs(save_path, exist_ok=True)
        for image_path in tqdm(image_paths):
            image = cv2.imread(image_path)
            result = self.predict_image(image, mode)
            cv2.imwrite(os.path.join(save_path, os.path.basename(image_path)), result)


In [45]:
model = Marmo("marmo.pt", "./sam_vit_h.pth")
image_paths = glob.glob("./test/marmoset-dlc-2021-05-07/labeled-data/reachingvideo1/*.png")

model.predict_images(image_paths, "./predict/", mode="box")

  0%|          | 0/2108 [00:00<?, ?it/s]


0: 640x640 1 marmoset, 3.8ms
Speed: 1.8ms preprocess, 3.8ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)


  0%|          | 1/2108 [00:00<18:01,  1.95it/s]


0: 480x640 1 marmoset, 4.5ms
Speed: 1.1ms preprocess, 4.5ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)


  0%|          | 2/2108 [00:00<16:42,  2.10it/s]

...


0: 480x640 1 marmoset, 4.2ms
Speed: 1.0ms preprocess, 4.2ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)


 59%|█████▊    | 1235/2108 [12:06<08:52,  1.64it/s]


0: 640x640 1 marmoset, 4.2ms
Speed: 1.8ms preprocess, 4.2ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


 59%|█████▊    | 1236/2108 [12:07<08:53,  1.64it/s]


0: 640x640 3 marmosets, 4.0ms
Speed: 1.4ms preprocess, 4.0ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)


In [30]:
!zip -r ./predict-one.zip ./predict && rm -rf ./predict

  adding: predict/ (stored 0%)
  adding: predict/img01497.png (deflated 3%)
  adding: predict/img01926.png (deflated 4%)
  adding: predict/img02443.png (deflated 4%)
  adding: predict/img02735.png (deflated 4%)
  adding: predict/img00807.png (deflated 5%)
  adding: predict/img04052.png (deflated 4%)
  adding: predict/img04441.png (deflated 4%)
  adding: predict/img03357.png (deflated 3%)
  adding: predict/img03744.png (deflated 3%)
  adding: predict/img06275.png (deflated 4%)
  adding: predict/img02416.png (deflated 4%)
  adding: predict/img02405.png (deflated 5%)
  adding: predict/img06949.png (deflated 7%)
  adding: predict/img01464.png (deflated 3%)
  adding: predict/img02022.png (deflated 4%)
  adding: predict/img04287.png (deflated 4%)
  adding: predict/img00999.png (deflated 3%)
  adding: predict/img00001.png (deflated 3%)
  adding: predict/img03968.png (deflated 4%)
  adding: predict/img02675.png (deflated 4%)
  adding: predict/img00953.png (deflated 4%)
  adding: predict/img003

Please make a pull request uploading predict-one.zip and predict-two.zip at https://github.com/sira-p/marmo-4b/tree/main